# Assignment 3 (Machine Learning)
> This assignment is solved by:-
* Name: Vishal Sehgal
* Group: CO11
* Roll Number: 102003265

### Question 1

> a

In [67]:
import numpy as np
import pandas as pd

In [68]:
df = pd.read_csv('./USA_Housing.csv')

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Avg. Area Income              5000 non-null   float64
 1   Avg. Area House Age           5000 non-null   float64
 2   Avg. Area Number of Rooms     5000 non-null   float64
 3   Avg. Area Number of Bedrooms  5000 non-null   float64
 4   Area Population               5000 non-null   float64
 5   Price                         5000 non-null   float64
dtypes: float64(6)
memory usage: 234.5 KB


In [70]:
X = df.drop('Price', axis=1)
y = df['Price']

> b

In [71]:
# Making use of z-score standard scaling.
def scaler(data):
    sigma = np.std(data)
    mu = np.mean(data)
    for i in range(len(data)):
        data[i] = (data[i] - mu)/sigma
    return data

In [72]:
X['Avg. Area House Age'] = scaler(X['Avg. Area House Age'])

In [73]:
X['Avg. Area Income'] = scaler(X['Avg. Area Income'])

In [74]:
X['Avg. Area Number of Bedrooms'] = scaler(X['Avg. Area Number of Bedrooms'])

In [75]:
X['Avg. Area Number of Rooms'] = scaler(X['Avg. Area Number of Rooms'])

In [76]:
X['Area Population'] = scaler(X['Area Population'])

In [77]:
X.head()

,Avg. Area Income,Avg. Area House Age,Avg. Area Number of Rooms,Avg. Area Number of Bedrooms,Area Population
0,1.028660,-0.296927,0.021274,0.088062,-1.317599
1,1.000808,0.025902,-0.255506,-0.722301,0.403999
2,-0.684629,-0.112303,1.516243,0.930840,0.072410
3,-0.491499,1.221572,-1.393077,-0.584540,-0.186734
4,-0.807073,-0.944834,0.846742,0.201513,-0.988387


> c

In [78]:
def _5folds(X, y):
    fold_size = len(X)/5
    data_split = []
    for i in range(5):
        data_split.append(int(fold_size*i))
    X1 = X[data_split[0]:data_split[1]]
    X2 = X[data_split[1]:data_split[2]]
    X3 = X[data_split[2]:data_split[3]]
    X4 = X[data_split[3]:data_split[4]]
    X5 = X[data_split[4]:len(X)]
    y1 = y[data_split[0]:data_split[1]]
    y2 = y[data_split[1]:data_split[2]]
    y3 = y[data_split[2]:data_split[3]]
    y4 = y[data_split[3]:data_split[4]]
    y5 = y[data_split[4]:len(X)]
    return np.array([X1, X2, X3, X4, X5]), np.array([y1, y2, y3, y4, y5])

In [79]:
X_fold, y_fold = _5folds(X, y)

In [80]:
X_fold

array([[[ 1.02865969, -0.29692705,  0.02127433,  0.08806222,
         -1.31759867],
        [ 1.00080775,  0.02590164, -0.25550611, -0.72230146,
          0.40399945],
        [-0.68462915, -0.11230283,  1.5162435 ,  0.93084045,
          0.07240989],
        ...,
        [-0.97390997,  0.32742189,  0.3856136 ,  1.15774228,
          1.0579596 ],
        [ 1.84495107,  0.5761329 ,  1.76755667, -0.50350326,
         -1.04502559],
        [ 0.01449665, -1.48943947,  1.45395689,  1.78172231,
         -0.19616035]],

       [[-1.14104906, -1.31776484,  1.11567765, -0.77092328,
          0.40225336],
        [-0.17876139, -2.08742238,  0.50539953,  0.84170044,
         -1.32320392],
        [ 0.27530504,  1.94518516,  0.73935604,  0.84170044,
          0.41067075],
        ...,
        [ 0.0830251 , -1.09969937,  0.35582492,  0.09616586,
          1.96852391],
        [ 1.02483129,  0.53248907,  0.56305663,  0.09616586,
          0.65632178],
        [ 0.48858196,  0.64329388, -0.72409425, 

> d

In [81]:
y_fold.shape

(5, 1000)

In [82]:
beta_mat = []
for i in range(len(X_fold)):
    ind = np.array([])
    for j in range(len(X_fold)):
        if(i!=j):
            ind = np.append(ind, j)
    X_test = X_fold[i]
    y_test = y_fold[i]
    if(i!=0):
        X_train = np.array(X_fold[0][0:])
        y_train = np.array(y_fold[0])
    else:
        X_train = np.array(X_fold[1][0:])
        y_train = np.array(y_fold[1])
    for j in ind:
        if((i==0) and (j==1)):
            continue
        if(j==0):
            continue
        X_train = np.vstack([X_train, X_fold[int(j)][0:]])
    for j in ind:
        if((i==0) and (j==1)):
            continue
        if(j==0):
            continue
        y_train = np.append(y_train, y_fold[int(j)])
    X_train = np.insert(X_train, 0, values=1, axis=1)
    A = X_train.T.dot(X_train)
    B = np.linalg.inv(A)
    C = B.dot(X_train.T)
    beta = C.dot(y_train)
    beta_mat.append(beta)
    print(f"\nFor Cross validation {i+1}: \n\nThe beta matrix is: \n{beta}")
    X_test = np.insert(X_test, 0, values=1, axis=1)
    y_predict = X_test.dot(beta)
    error=y_test-y_predict
    square_error=np.power(error,2)
    sum_square_error=np.sum(square_error)
    mean_square_error=sum_square_error/len(y_predict)
    rms_error=np.sqrt(mean_square_error)
    y_mean=np.mean(y_test)
    total_variance=np.sum((y_test-y_mean)**2)
    print(f"\nR2-Score: {1-sum_square_error/total_variance}")


For Cross validation 1: 

The beta matrix is: 
[1231872.07704613  229923.70318635  164052.98579011  120923.57426579
    3533.77470973  151855.58542488]

R2-Score: 0.9175899480765107

For Cross validation 2: 

The beta matrix is: 
[1232586.40453402  228365.32287054  165201.14881927  122539.46855211
    1697.03433458  150055.81710562]

R2-Score: 0.920301549640113

For Cross validation 3: 

The beta matrix is: 
[1231448.09063372  230730.20810582  163554.55706976  121669.08713233
    2620.25215054  150923.12489298]

R2-Score: 0.9152429915320014

For Cross validation 4: 

The beta matrix is: 
[1.23144707e+06 2.29921558e+05 1.64523054e+05 1.19737507e+05
 1.12425659e+03 1.51317802e+05]

R2-Score: 0.9208503836977655

For Cross validation 5: 

The beta matrix is: 
[1233028.38536437  230778.59921985  163755.13607252  121885.1351222
    1247.01304382  150084.29759618]

R2-Score: 0.9138111758717495


> e

In [83]:
# We found that the best value for R2 is in 4th cross validation and hence having the best value of beta in it.
beta_mat[3]

array([1.23144707e+06, 2.29921558e+05, 1.64523054e+05, 1.19737507e+05,
       1.12425659e+03, 1.51317802e+05])

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [85]:
X_test = np.array(X_test)
X_test = np.insert(X_test, 0, values=1, axis=1)

In [86]:
y_predict = X_test.dot(beta_mat[3])

In [87]:
error=y_test-y_predict
square_error=np.power(error,2)
sum_square_error=np.sum(square_error)
mean_square_error=sum_square_error/len(y_predict)
rms_error=np.sqrt(mean_square_error)
y_mean=np.mean(y_test)
total_variance=np.sum((y_test-y_mean)**2)
print(f"\nR2-Score: {1-sum_square_error/total_variance}")


R2-Score: 0.9146868482147401


### Question 2

In [88]:
from sklearn.model_selection import train_test_split

In [89]:
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [90]:
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, test_size=0.2, random_state=42)

In [91]:
len(X_train), len(X_val), len(X_test)

(2800, 700, 1500)

In [92]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [93]:
learning_rates = [0.001, 0.01, 0.1, 1]
beta_mat = []
for lr in learning_rates:
    beta=np.zeros(6)
    number_of_iterations=1000
    for i in range(number_of_iterations):
        x0_gradient=0
        x1_gradient=0
        x2_gradient=0
        x3_gradient=0
        x4_gradient=0
        x5_gradient=0    
        for j in range(len(X_train)):
            a=X_train[j,0]
            b=X_train[j,1]
            c=X_train[j,2]
            d=X_train[j,3]
            e=X_train[j,4]
            f=y_train[j]
            x0_gradient+=(beta[0]+(beta[1]*a)+(beta[2]*b)+(beta[3]*c)+(beta[4]*d)+(beta[5]*e)-f)
            x1_gradient+=((beta[0]+(beta[1]*a)+(beta[2]*b)+(beta[3]*c)+(beta[4]*d)+(beta[5]*e)-f)*a)
            x2_gradient+=((beta[0]+(beta[1]*a)+(beta[2]*b)+(beta[3]*c)+(beta[4]*d)+(beta[5]*e)-f)*b)
            x3_gradient+=((beta[0]+(beta[1]*a)+(beta[2]*b)+(beta[3]*c)+(beta[4]*d)+(beta[5]*e)-f)*c)
            x4_gradient+=((beta[0]+(beta[1]*a)+(beta[2]*b)+(beta[3]*c)+(beta[4]*d)+(beta[5]*e)-f)*d)
            x5_gradient+=((beta[0]+(beta[1]*a)+(beta[2]*b)+(beta[3]*c)+(beta[4]*d)+(beta[5]*e)-f)*e)
        beta[0]=beta[0]-lr/len(X_train)*x0_gradient
        beta[1]=beta[1]-lr/len(X_train)*x1_gradient
        beta[2]=beta[2]-lr/len(X_train)*x2_gradient
        beta[3]=beta[3]-lr/len(X_train)*x3_gradient
        beta[4]=beta[4]-lr/len(X_train)*x4_gradient
        beta[5]=beta[5]-lr/len(X_train)*x5_gradient
    beta_mat.append(beta)
    print(beta)

[778829.54135369 145453.10988561 101705.72638778  81088.52016625
  32930.95853355  88928.37718238]
[1232562.5125492   230048.76664688  163686.93503606  121406.94107918
    3117.47363933  150655.97459714]
[1232618.32011842  230067.9889464   163710.33259401  121681.42752283
    2832.15066521  150657.52262836]
[1232618.32011842  230067.9889464   163710.33259401  121681.42752283
    2832.15066521  150657.52262836]


In [94]:
X_val = np.array(X_val)
y_val = np.array(y_val)
X_test = np.array(X_test)
y_test = np.array(y_test)
X_val=np.insert(X_val,0,values=1,axis=1)
X_test=np.insert(X_test,0,values=1,axis=1)

In [96]:
from sklearn.metrics import r2_score

In [98]:
for beta in beta_mat:
    y_predict_val=X_val.dot(beta)
    y_predict_test=X_test.dot(beta)
    print(f"For beta = {beta}, \nR2-score (Validation) is {r2_score(y_val, y_predict_val)} \nR2-score (Testing) is {r2_score(y_test, y_predict_test)}\n")

For beta = [778829.54135369 145453.10988561 101705.72638778  81088.52016625
  32930.95853355  88928.37718238], 
R2-score (Validation) is -0.8125204084681943 
R2-score (Testing) is -0.9913653412489318

For beta = [1232562.5125492   230048.76664688  163686.93503606  121406.94107918
    3117.47363933  150655.97459714], 
R2-score (Validation) is 0.9098183094422969 
R2-score (Testing) is 0.9147434800538763

For beta = [1232618.32011842  230067.9889464   163710.33259401  121681.42752283
    2832.15066521  150657.52262836], 
R2-score (Validation) is 0.9097995626742027 
R2-score (Testing) is 0.9147570103083724

For beta = [1232618.32011842  230067.9889464   163710.33259401  121681.42752283
    2832.15066521  150657.52262836], 
R2-score (Validation) is 0.9097995626742028 
R2-score (Testing) is 0.9147570103083724



In [99]:
# Hence the best test score value is for lr=0.1 and lr=1 so ans becomes
print(f"Best beta values are: {beta_mat[2]}")

Best beta values are: [1232618.32011842  230067.9889464   163710.33259401  121681.42752283
    2832.15066521  150657.52262836]


### Question 3

> 1.

In [155]:
df = pd.read_csv('./imports-85.data', names=["symboling", "normalized_losses", "make", "fuel_type", "aspiration","num_doors", "body_style", "drive_wheels", "engine_location", "wheel_base", "length", "width", "height", "curb_weight", "engine_type", "num_cylinders", "engine_size", "fuel_system", "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm", "city_mpg", "highway_mpg", "price"])

In [156]:
df = df.replace("?", np.nan)

In [157]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized_losses  164 non-null    object 
 2   make               205 non-null    object 
 3   fuel_type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num_doors          203 non-null    object 
 6   body_style         205 non-null    object 
 7   drive_wheels       205 non-null    object 
 8   engine_location    205 non-null    object 
 9   wheel_base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb_weight        205 non-null    int64  
 14  engine_type        205 non-null    object 
 15  num_cylinders      205 non-null    object 
 16  engine_size        205 non

> 2.

In [158]:
df.head()

,symboling,normalized_losses,make,fuel_type,aspiration,num_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [159]:
df['normalized_losses'] = df['normalized_losses'].astype(float)

In [160]:
df['stroke'] = df['stroke'].astype(float)

In [161]:
df['horsepower'] = df['horsepower'].astype(float)

In [162]:
df['peak_rpm'] = df['peak_rpm'].astype(float)

In [163]:
df['price'] = df['price'].astype(float)

In [164]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized_losses  164 non-null    float64
 2   make               205 non-null    object 
 3   fuel_type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num_doors          203 non-null    object 
 6   body_style         205 non-null    object 
 7   drive_wheels       205 non-null    object 
 8   engine_location    205 non-null    object 
 9   wheel_base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb_weight        205 non-null    int64  
 14  engine_type        205 non-null    object 
 15  num_cylinders      205 non-null    object 
 16  engine_size        205 non

In [165]:
df = df[df['price'].isnull()==False]

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          201 non-null    int64  
 1   normalized_losses  164 non-null    float64
 2   make               201 non-null    object 
 3   fuel_type          201 non-null    object 
 4   aspiration         201 non-null    object 
 5   num_doors          199 non-null    object 
 6   body_style         201 non-null    object 
 7   drive_wheels       201 non-null    object 
 8   engine_location    201 non-null    object 
 9   wheel_base         201 non-null    float64
 10  length             201 non-null    float64
 11  width              201 non-null    float64
 12  height             201 non-null    float64
 13  curb_weight        201 non-null    int64  
 14  engine_type        201 non-null    object 
 15  num_cylinders      201 non-null    object 
 16  engine_size        201 non

In [167]:
df['normalized_losses'].fillna(df['normalized_losses'].median(), inplace=True)

In [169]:
df['bore'].fillna(df['bore'].median(), inplace=True)

In [170]:
df['stroke'].fillna(df['stroke'].median(), inplace=True)

In [172]:
df['horsepower'].fillna(df['horsepower'].median(), inplace=True)

In [174]:
df['peak_rpm'].fillna(df['peak_rpm'].median(), inplace=True)

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          201 non-null    int64  
 1   normalized_losses  201 non-null    float64
 2   make               201 non-null    object 
 3   fuel_type          201 non-null    object 
 4   aspiration         201 non-null    object 
 5   num_doors          199 non-null    object 
 6   body_style         201 non-null    object 
 7   drive_wheels       201 non-null    object 
 8   engine_location    201 non-null    object 
 9   wheel_base         201 non-null    float64
 10  length             201 non-null    float64
 11  width              201 non-null    float64
 12  height             201 non-null    float64
 13  curb_weight        201 non-null    int64  
 14  engine_type        201 non-null    object 
 15  num_cylinders      201 non-null    object 
 16  engine_size        201 non

> 3. 

In [179]:
# i.

In [177]:
df['num_doors'].unique()

array(['two', 'four', nan], dtype=object)

In [182]:
df['num_doors'] = df['num_doors'].replace('two', 2)
df['num_doors'] = df['num_doors'].replace('four', 4)
df['num_doors'] = df['num_doors'].astype(float)

In [183]:
df['num_doors'].fillna(df['num_doors'].median(), inplace=True)

In [185]:
df['num_cylinders'].unique()

array(['four', 'six', 'five', 'three', 'twelve', 'two', 'eight'],
      dtype=object)

In [186]:
df['num_cylinders'] = df['num_cylinders'].replace('four', 4)
df['num_cylinders'] = df['num_cylinders'].replace('six', 6)
df['num_cylinders'] = df['num_cylinders'].replace('five', 5)
df['num_cylinders'] = df['num_cylinders'].replace('three', 3)
df['num_cylinders'] = df['num_cylinders'].replace('twelve', 12)
df['num_cylinders'] = df['num_cylinders'].replace('two', 2)
df['num_cylinders'] = df['num_cylinders'].replace('eight', 8)
df['num_cylinders'] = df['num_cylinders'].astype(float)

In [187]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          201 non-null    int64  
 1   normalized_losses  201 non-null    float64
 2   make               201 non-null    object 
 3   fuel_type          201 non-null    object 
 4   aspiration         201 non-null    object 
 5   num_doors          201 non-null    float64
 6   body_style         201 non-null    object 
 7   drive_wheels       201 non-null    object 
 8   engine_location    201 non-null    object 
 9   wheel_base         201 non-null    float64
 10  length             201 non-null    float64
 11  width              201 non-null    float64
 12  height             201 non-null    float64
 13  curb_weight        201 non-null    int64  
 14  engine_type        201 non-null    object 
 15  num_cylinders      201 non-null    float64
 16  engine_size        201 non

In [188]:
# ii.

In [194]:
df = pd.concat([df, pd.get_dummies(df['body_style'], drop_first=True)], axis=1)

In [196]:
df = pd.concat([df, pd.get_dummies(df['drive_wheels'], drop_first=True)], axis=1)

In [197]:
df.drop(['body_style', 'drive_wheels'], axis=1, inplace=True)

In [198]:
# iii.

In [199]:
from sklearn.preprocessing import LabelEncoder

In [201]:
label_encoder = LabelEncoder()
df['make'] = label_encoder.fit_transform(df['make'])

In [203]:
label_encoder = LabelEncoder()
df['aspiration'] = label_encoder.fit_transform(df['aspiration'])

In [204]:
label_encoder = LabelEncoder()
df['engine_location'] = label_encoder.fit_transform(df['engine_location'])

In [205]:
label_encoder = LabelEncoder()
df['fuel_type'] = label_encoder.fit_transform(df['fuel_type'])

In [208]:
# iv.

In [209]:
df['fuel_system'].unique()

array(['mpfi', '2bbl', 'mfi', '1bbl', 'spfi', '4bbl', 'idi', 'spdi'],
      dtype=object)

In [211]:
df['fuel_system'] = df['fuel_system'].replace('mpfi', 1)

In [219]:
df.loc[df['fuel_system']!=1, 'fuel_system'] = 0

In [220]:
df['fuel_system'].unique()

array([1, 0], dtype=object)

In [227]:
df['fuel_system'] = df['fuel_system'].astype(float)

In [221]:
# v.

In [222]:
df['engine_type'].unique()

array(['dohc', 'ohcv', 'ohc', 'l', 'rotor', 'ohcf'], dtype=object)

In [223]:
df['engine_type'] = df['engine_type'].replace('ohc', 1)

In [224]:
df.loc[df['engine_type']!=1, 'engine_type'] = 0

In [225]:
df['engine_type'].unique()

array([0, 1], dtype=object)

In [228]:
df['engine_type'] = df['engine_type'].astype(float)

In [232]:
df['bore'] = df['bore'].astype(float)

In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201 entries, 0 to 204
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          201 non-null    int64  
 1   normalized_losses  201 non-null    float64
 2   make               201 non-null    int64  
 3   fuel_type          201 non-null    int64  
 4   aspiration         201 non-null    int64  
 5   num_doors          201 non-null    float64
 6   engine_location    201 non-null    int64  
 7   wheel_base         201 non-null    float64
 8   length             201 non-null    float64
 9   width              201 non-null    float64
 10  height             201 non-null    float64
 11  curb_weight        201 non-null    int64  
 12  engine_type        201 non-null    float64
 13  num_cylinders      201 non-null    float64
 14  engine_size        201 non-null    int64  
 15  fuel_system        201 non-null    float64
 16  bore               201 non

> 4.

In [234]:
X = df.drop('price', axis=1)
y = df['price']

> 5.

In [236]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [237]:
from sklearn.linear_model import LinearRegression

In [238]:
model = LinearRegression()

In [239]:
model.fit(X_train, y_train)

LinearRegression()

In [240]:
pred = model.predict(X_test)

In [241]:
print(r2_score(y_test, pred))

0.8770360259612894


> 6.

In [252]:
from sklearn.decomposition import PCA

In [285]:
for i in range(2, 29):
    pca = PCA(n_components=i)
    pca.fit(X)
    X_red = pca.transform(X)
    X_train_red, X_test_red, y_train, y_test = train_test_split(X_red, y, test_size=0.3, random_state=42)
    model_2 = LinearRegression()
    model_2.fit(X_train_red, y_train)
    pred = model_2.predict(X_test_red)
    print(f"Number of Components: {i}\nR2Score: {r2_score(y_test, pred)}\n")

Number of Components: 2
R2Score: 0.7037665562892313

Number of Components: 3
R2Score: 0.6937050628322967

Number of Components: 4
R2Score: 0.7824369800050526

Number of Components: 5
R2Score: 0.7895323833783442

Number of Components: 6
R2Score: 0.78195893273085

Number of Components: 7
R2Score: 0.7733839620778229

Number of Components: 8
R2Score: 0.7875221421053044

Number of Components: 9
R2Score: 0.7911090339624935

Number of Components: 10
R2Score: 0.7910220801730004

Number of Components: 11
R2Score: 0.7932729360151403

Number of Components: 12
R2Score: 0.7974466760923038

Number of Components: 13
R2Score: 0.799484081883667

Number of Components: 14
R2Score: 0.8007210876158104

Number of Components: 15
R2Score: 0.7985247154761899

Number of Components: 16
R2Score: 0.8098846517749281

Number of Components: 17
R2Score: 0.8286467741978211

Number of Components: 18
R2Score: 0.828310387182239

Number of Components: 19
R2Score: 0.8296663968532649

Number of Components: 20
R2Score: 0.8354

In [286]:
# R2 Score is only increased by decreasing Number of components from 29 to 28 & 27 only.